In [ ]:
import random  # Import random module to generate random choices
import copy  # Import copy module to make deep copies of game states
import math  # Import math module for mathematical operations
import time  # Import time module to measure elapsed time

# Class representing the Dots and Boxes game board
class DotsAndBoxes:
    def __init__(self, num_rows, num_columns):
        # Initialize the game state with dictionaries for edge status and box ownership
        self.edge_status = {}  # Dictionary to store all possible edges and their status (0 for unplayed, 1 for played)
        self.box_ownership = {}  # Dictionary to store the boxes and which player has claimed them
        self.num_rows = num_rows  # Number of rows in the grid
        self.num_columns = num_columns  # Number of columns in the grid
        self.player_a_score = 0  # Player A's score
        self.player_b_score = 0  # Player B's score

        # Create all possible horizontal edges
        for row in range(num_rows):
            for col in range(num_columns - 1):
                # Each horizontal edge connects two adjacent points horizontally
                self.edge_status[((col + (row * num_columns)), col + (row * num_columns) + 1)] = 0

        # Create all possible vertical edges
        for row in range(num_rows - 1):
            for col in range(num_columns):
                # Each vertical edge connects two adjacent points vertically
                self.edge_status[(col + (row * num_columns), col + num_columns + (row * num_columns))] = 0

        # Create boxes and initialize their ownership
        for row in range(num_rows - 1):
            for col in range(num_columns - 1):
                # Each box is defined by four sides (top, left, right, bottom)
                box = [(col + row * num_columns, col + row * num_columns + 1)]
                box.append((box[0][0], box[0][1] + num_columns - 1))
                box.append((box[0][0] + 1, box[0][1] + num_columns))
                box.append((box[0][0] + num_columns, box[2][1]))
                self.box_ownership[tuple(box)] = 0  # Initialize each box's ownership to 0 (unclaimed)

    def render_row(self, row_idx):
        # Render a row of the grid, displaying the points and horizontal lines
        left = (row_idx * self.num_columns)
        right = left + 1
        for col in range(self.num_columns - 1):
            if self.edge_status[(left, right)] == 0:
                print("{:^3d}".format(left), end="     ")
            else:
                print("{:^3d}  ——".format(left), end=" ")
            left = right
            right = left + 1
        print("{:^3d}".format(left))

    def render_vertical_edge(self, upper_left, lower_left):
        # Render a vertical line between two points if the play has been made
        if self.edge_status[(upper_left, lower_left)] == 0:
            print("    ", end=" ")
        else:
            print(" |", end="   ")

    def render_middle_row(self, row_idx):
        # Render a row of boxes, including vertical lines and scores if the box is claimed
        upper_left = (row_idx * self.num_columns)
        upper_right = upper_left + 1
        lower_left = upper_left + self.num_columns
        lower_right = lower_left + 1

        for col in range(self.num_columns - 1):
            self.render_vertical_edge(upper_left, lower_left)
            top = (upper_left, upper_right)
            left = (upper_left, lower_left)
            right = (upper_right, lower_right)
            bottom = (lower_left, lower_right)
            ownership = self.box_ownership[(top, left, right, bottom)]

            if ownership == 0:
                print("  ", end=" ")
            else:
                print(" " + ownership, end=" ")

            upper_left, lower_left = upper_right, lower_right
            upper_right += 1
            lower_right += 1
        self.render_vertical_edge(upper_left, lower_left)
        print()

    def render_board(self):
        # Render the entire game board, including scores
        for row in range(self.num_rows - 1):
            self.render_row(row)
            self.render_middle_row(row)
        self.render_row(self.num_rows - 1)
        print("\nPlayerA (you): {}\nPlayerB (AI): {}\n--------------------------------------".format(self.player_a_score, self.player_b_score))

    def check_for_completed_boxes(self, is_player_a):
        # Check if any boxes have been completed by the current play
        current_player = "A" if is_player_a else "B"
        completed_edges = {edge for edge in self.edge_status if self.edge_status[edge] == 1}
        incomplete_boxes = [box for box in self.box_ownership if self.box_ownership[box] == 0]
        new_scores = 0

        for box in incomplete_boxes:
            if set(box).issubset(completed_edges):
                new_scores += 1
                self.box_ownership[box] = current_player
        return new_scores

    def make_move(self, start_point, end_point, is_player_a):
        # Make a move if it is valid, update the game state, and check for any new scores
        try:
            if self.edge_status[(start_point, end_point)] == 1:
                return False
        except KeyError:
            return False

        self.edge_status[(start_point, end_point)] = 1
        new_scores = self.check_for_completed_boxes(is_player_a)
        if is_player_a:
            self.player_a_score += new_scores
        else:
            self.player_b_score += new_scores
        return True

    def get_available_moves(self):
        # Get a list of all available moves (unplayed edges)
        return [edge for edge in self.edge_status if self.edge_status[edge] == 0]

    def is_game_over(self):
        # Check if the game is over (all boxes have been claimed)
        return self.player_a_score + self.player_b_score == len(self.box_ownership)


# Class representing a player who makes random moves
class RandomPlayer:
    def __init__(self, is_player_a):
        # Initialize the player as either Player A or Player B
        self.is_player_a = is_player_a

    def make_move(self, game):
        # Randomly select an available move and make the play
        available_moves = game.get_available_moves()
        move = random.choice(available_moves)
        game.make_move(*move, self.is_player_a)
        player = "You" if self.is_player_a else "AI"
        print("{}'s move: {} {}\n".format(player, *move))


# Class representing a human player
class HumanPlayer:
    def __init__(self, is_player_a):
        # Initialize the player as either Player A or Player B and set the player name
        self.is_player_a = is_player_a
        self.player_name = "You" if is_player_a else "AI"

    def make_move(self, game):
        # Prompt the human player for their move and validate the input
        while True:
            move = input("The turn has passed to {}, make your move (AS 0 1): ".format(self.player_name))
            move = move.split()
            try:
                move[0], move[1] = int(move[0]), int(move[1])
                if len(move) != 2:
                    print("You must enter a start point end point, try again")
                    continue
            except (NameError, IndexError, ValueError):
                print("You must enter a start point end point, try again")
                continue
            move.sort()
            valid_move = game.make_move(*move, self.is_player_a)
            if valid_move:
                return
            print("That move does not exist, try again")


# Class representing a player who uses the alpha-beta pruning algorithm
class AlphaBetaPlayer:
    def __init__(self, is_player_a):
        # Initialize the player as either Player A or Player B
        self.is_player_a = is_player_a

    def alpha_beta_search(self, game, move, depth, alpha, beta, is_maximizing_player):
        # Alpha-beta pruning algorithm to decide the best play
        if game.is_game_over() or depth == 0:
            # Base case: if the game is over or the maximum depth is reached, return the score difference
            return game.player_a_score - game.player_b_score, move

        if is_maximizing_player:
            # Maximizing player (Player A)
            max_eval = -math.inf  # Initialize the value to negative infinity
            best_move = move
            for next_move in game.get_available_moves():
                # Iterate through all available moves
                new_game = copy.deepcopy(game)  # Create a deep copy of the game to simulate the move
                old_score = new_game.player_a_score  # Store the old score before making the move
                new_game.make_move(*next_move, True)  # Make the move for Player A
                new_score = new_game.player_a_score  # Get the new score after making the move

                if new_score == old_score:
                    # If the score didn't change, it's the next player's turn (Player B)
                    eval, _ = self.alpha_beta_search(new_game, next_move, depth - 1, alpha, beta, False)
                else:
                    # If the score changed, it's still Player A's turn
                    eval, _ = self.alpha_beta_search(new_game, next_move, depth - 1, alpha, beta, True)

                if eval > max_eval:
                    # If the current evaluation is better (greater) than the max evaluation, update the best move and max evaluation
                    best_move = next_move
                    max_eval = eval

                alpha = max(alpha, eval)  # Update alpha value
                if alpha >= beta:
                    # If alpha is greater than or equal to beta, prune the remaining branches
                    break
            return max_eval, best_move

        else:
            # Minimizing player (Player B)
            min_eval = math.inf  # Initialize the value to positive infinity
            best_move = move
            for next_move in game.get_available_moves():
                # Iterate through all available moves
                new_game = copy.deepcopy(game)  # Create a deep copy of the game to simulate the move
                old_score = new_game.player_b_score  # Store the old score before making the move
                new_game.make_move(*next_move, False)  # Make the move for Player B
                new_score = new_game.player_b_score  # Get the new score after making the move

                if new_score == old_score:
                    # If the score didn't change, it's the next player's turn (Player A)
                    eval, _ = self.alpha_beta_search(new_game, next_move, depth - 1, alpha, beta, True)
                else:
                    # If the score changed, it's still Player B's turn
                    eval, _ = self.alpha_beta_search(new_game, next_move, depth - 1, alpha, beta, False)

                if eval < min_eval:
                    # If the current evaluation is better (less) than the min evaluation, update the best move and min evaluation
                    best_move = next_move
                    min_eval = eval

                beta = min(beta, eval)  # Update beta value
                if beta <= alpha:
                    # If beta is less than or equal to alpha, prune the remaining branches
                    break
            return min_eval, best_move

    def make_move(self, game):
        # Make a play using the alpha-beta pruning algorithm
        start_time = time.time()  # Record the start time to measure elapsed time
        available_moves = game.get_available_moves()  # Get the number of open plays

        if len(available_moves) == 1:
            # If there's only one open play, make that play
            best_move = available_moves[0]
            game.make_move(*best_move, self.is_player_a)
            return

        depth = math.floor(math.log(19000, len(available_moves)))  # Determine the search depth based on the size of play space
        _, best_move = self.alpha_beta_search(game, (0, 0), depth, -math.inf, math.inf, self.is_player_a)  # Get the best play using alpha-beta pruning
        elapsed_time = time.time() - start_time  # Calculate the time elapsed to make the move

        if best_move == (0, 0):
            # If no valid play was found, choose a random play
            best_move = random.choice(available_moves)
        game.make_move(*best_move, self.is_player_a)

        player = "A" if self.is_player_a else "B"
        print("Player {}'s move: {} {}".format(player, *best_move))
        print("Time elapsed to make move: {:.2f} seconds".format(elapsed_time))


# Class representing the game management and flow
class DotsAndBoxesGame:
    def __init__(self, player_a_type="random", player_b_type="random", num_rows=5, num_columns=5):
        # Initialize the game with the specified types of players and grid dimensions
        self.num_rows = num_rows  # Number of rows for the game grid
        self.num_columns = num_columns  # Number of columns for the game grid

        # Initialize Player A based on the specified type
        if player_a_type == "random":
            self.player_a = RandomPlayer(True)  # Create a RandomPlayer for Player A
        elif player_a_type == "alphabeta":
            self.player_a = AlphaBetaPlayer(True)  # Create an AlphaBetaPlayer for Player A
        else:
            self.player_a = HumanPlayer(True)  # Create a HumanPlayer for Player A

        # Initialize Player B based on the specified type
        if player_b_type == "random":
            self.player_b = RandomPlayer(False)  # Create a RandomPlayer for Player B
        elif player_b_type == "alphabeta":
            self.player_b = AlphaBetaPlayer(False)  # Create an AlphaBetaPlayer for Player B
        else:
            self.player_b = HumanPlayer(False)  # Create a HumanPlayer for Player B

    def start_game(self):
        # Start and play the game
        game_board = DotsAndBoxes(self.num_rows, self.num_columns)  # Create a new game board
        print()
        game_board.render_board()  # Render the initial game board
        print()

        coin_toss = random.randint(1, 2)  # Toss a coin to decide who goes first
        print("The coin landed on {}".format("HEAD" if coin_toss == 1 else "TAIL"))
        print("{} will start".format("You (player A)" if coin_toss == 1 else "AI (player B)"))
        print()

        while not game_board.is_game_over():
            # Loop until the game is over
            while not game_board.is_game_over():
                if coin_toss == 2:
                    # If Player B won the coin toss, switch to Player A's turn
                    coin_toss = 3
                    break
                previous_score = game_board.player_a_score  # Store the old score of Player A
                self.player_a.make_move(game_board)  # Player A makes a move
                game_board.render_board()  # Render the game board after Player A's move
                if previous_score == game_board.player_a_score:
                    # If Player A's score didn't change, switch to Player B's turn
                    break

            while not game_board.is_game_over():
                previous_score = game_board.player_b_score  # Store the old score of Player B
                self.player_b.make_move(game_board)  # Player B makes a move
                game_board.render_board()  # Render the game board after Player B's move
                if previous_score == game_board.player_b_score:
                    # If Player B's score didn't change, switch to Player A's turn
                    break

        # Determine and announce the winner
        if game_board.player_a_score == game_board.player_b_score:
            print("It's a TIE!")
        elif game_board.player_a_score > game_board.player_b_score:
            print("Congrats, YOU win!")
        else:
            print("Hard luck, AI wins!")


def main():
    # Main function to start the game
    print("""
  _____        _                  ____
 |  __ \      | |         ___    |  _ \\
 | |  | | ___ | |_ ___   ( _ )   | |_) | _____  _____  ___
 | |  | |/ _ \| __/ __|  / _ \/\ |  _ < / _ \ \/ / _ \/ __|
 | |__| | (_) | |_\__ \ | (_>  < | |_) | (_) >  <  __/\__ \\
 |_____/ \___/ \__|___/  \___/\/ |____/ \___/_/\_\___||___/

    """)

    print("\n\n**************************************************************************************************")
    print("××××××××××                          WELCOME TO DOTS & BOXES!!                           ××××××××××")
    print("**********  YOU CAN COUNT ON US TO GIVE YOU THE FINEST EXPERIENCE WHILE PLAYING.        **********")
    print("××××××××××                OUR GOAL IS TO ENSURE YOU HAVE A GREAT TIME                   ××××××××××")
    print("**************************************************************************************************\n")
    print("To learn more about the game click here: https://shorturl.at/6mDgC\n")

    play_again = "y"  # Variable to control the game loop
    while play_again == "y":
        # Prompt the user to choose the type of opponent
        opponent_type = input("Enter (r) to play against a random player\n"
                              "Enter (i) for a more intelligent one(Intelligent takes player a few seconds to think)\nYour opponent is: ").lower()

        if opponent_type not in ("r", "i"):
            print("You have to write either (i) or (r), Please try again")
            continue

        player_b_type = "alphabeta" if opponent_type == "i" else "random"

        while True:
            # Prompt the user to choose the number of rows for the grid
            num_rows = input("How many rows you want on the board game? (maximum 10): ")
            try:
                num_rows = int(num_rows)
                if num_rows < 2:
                    print("Number of rows must be more than 1")
                    continue
                if num_rows > 10:
                    print("Too many rows! Try another number not exceeding 10")
                    continue
            except ValueError:
                print("It has to be an integer value! Try again please.")
                continue
            break

        while True:
            # Prompt the user to choose the number of columns for the grid
            num_columns = input("How many columns you want on the board game? (maximum 10): ")
            try:
                num_columns = int(num_columns)
                if num_columns < 2:
                    print("Number of columns must be more than 1")
                    continue
                if num_columns > 10:
                    print("Too many columns! Try another number not exceeding 10")
                    continue
            except ValueError:
                print("It has to be an integer value! Try again please.")
                continue
            break

        # Initialize the game with the chosen grid size and opponent type
        game_instance = DotsAndBoxesGame("Human", player_b_type, num_rows, num_columns)
        game_instance.start_game()  # Start the game

        # Prompt the user to play again
        play_again = input("Do you want to play again? (y/n): ").lower()

    print("Thank you for playing, \nWe hope it was an enjoyable game")


# Check if the script is run directly (not imported as a module)
if __name__ == "__main__":
    main()




  _____        _                  ____
 |  __ \      | |         ___    |  _ \
 | |  | | ___ | |_ ___   ( _ )   | |_) | _____  _____  ___
 | |  | |/ _ \| __/ __|  / _ \/\ |  _ < / _ \ \/ / _ \/ __|
 | |__| | (_) | |_\__ \ | (_>  < | |_) | (_) >  <  __/\__ \
 |_____/ \___/ \__|___/  \___/\/ |____/ \___/_/\_\___||___/

    


**************************************************************************************************
××××××××××                          WELCOME TO DOTS & BOXES!!                           ××××××××××
**********  YOU CAN COUNT ON US TO GIVE YOU THE FINEST EXPERIENCE WHILE PLAYING.        **********
××××××××××                OUR GOAL IS TO ENSURE YOU HAVE A GREAT TIME                   ××××××××××
**************************************************************************************************

To learn more about the game click here: https://shorturl.at/6mDgC

Enter (r) to play against a random player
Enter (i) for a more intelligent one(Intelligent takes player a few